# ES Order Move to BBF BANs (v2)

**v2 FIX:** Replaced pd.read_sql_query() with cursor.fetchall() pattern to fix psycopg2 incompatibility

**PREREQUISITE:** BAN creation must be completed first (new -BBF BANs must exist)

## Purpose
Move orders to new BBF BANs by updating:
1. **SF:** `Order.Billing_Invoice__c` → new -BBF BAN SFID
2. **OSS:** `orders.account_id` → new OSS account_id

## What Gets Moved
- Orders WITHOUT "PA MARKET DECOM" in Project_Group__c
- That are on a BAN which has a new -BBF counterpart

## Safety
- `DRY_RUN = True` by default
- Dry run outputs Excel showing what would be updated
- Live run performs updates and outputs results

In [1]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from simple_salesforce import Salesforce
import time

# Active statuses
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# WD identifier - orders with this stay on original BAN
WD_PROJECT_GROUP = "PA MARKET DECOM"

# ES Salesforce credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"

# Connection credentials - Heroku
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

# Connection credentials - OSS
oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

print("Connecting to Heroku...")
conn = psycopg2.connect(**heroku_conn_kwargs)
print("✅ Connected to Heroku")

print("Connecting to OSS...")
oconn = psycopg2.connect(**oss_conn_kwargs)
print("✅ Connected to OSS")

print("Connecting to ES Salesforce...")
es_sf = Salesforce(username=ES_USERNAME, password=ES_PASSWORD, security_token=ES_TOKEN)
print("✅ Connected to ES Salesforce")

Connecting to Heroku...
✅ Connected to Heroku
Connecting to OSS...
✅ Connected to OSS
Connecting to ES Salesforce...
✅ Connected to ES Salesforce


In [2]:
# === CONFIGURATION ===
DRY_RUN = True  # Set to False to actually update orders
OUTPUT_DIR = "."

# Output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"{OUTPUT_DIR}/es_order_move_to_bbf_ban_v2_{run_type}_{timestamp}.xlsx"

print(f"DRY_RUN: {DRY_RUN}")
print(f"Output file: {output_file}")

if not DRY_RUN:
    print("\n⚠️  WARNING: LIVE RUN - Orders will be updated!")
    print("    Press Ctrl+C now to abort if this is not intended.")

DRY_RUN: True
Output file: ./es_order_move_to_bbf_ban_v2_dry_run_20260112_105344.xlsx


In [3]:
# === STEP 1: Get all new -BBF BANs ===

bbf_bans_sql = """
SELECT 
    sfid AS new_ban_sfid,
    name AS new_ban_name,
    account_number__c AS new_account_id,
    legacy_es_id__c AS old_ban_sfid,
    account__c AS customer_sfid,
    account_name__c,
    bbf_ban__c
FROM sfprod.billing_invoice__c
WHERE legacy_es_id__c IS NOT NULL
  AND bbf_ban__c = true
ORDER BY name;
"""

print("Querying new -BBF BANs...")
with conn.cursor() as cur:
    cur.execute(bbf_bans_sql)
    bbf_bans_rows = cur.fetchall()

bbf_bans_df = pd.DataFrame(bbf_bans_rows)
print(f"✅ Found {len(bbf_bans_df)} new -BBF BANs")

if len(bbf_bans_df) == 0:
    raise Exception("❌ ERROR: No -BBF BANs found! BAN creation must be run first.")

print(f"\nSample -BBF BANs:")
print(
    bbf_bans_df[
        ["new_ban_name", "new_account_id", "old_ban_sfid", "account_name__c"]
    ].head(10)
)

# Build mapping dictionaries
old_to_new_ban_sfid = dict(
    zip(bbf_bans_df["old_ban_sfid"], bbf_bans_df["new_ban_sfid"])
)
old_to_new_ban_name = dict(
    zip(bbf_bans_df["old_ban_sfid"], bbf_bans_df["new_ban_name"])
)
old_to_new_account_id = dict(
    zip(bbf_bans_df["old_ban_sfid"], bbf_bans_df["new_account_id"])
)

print(f"\nBuilt mapping for {len(old_to_new_ban_sfid)} BANs")

Querying new -BBF BANs...
✅ Found 2505 new -BBF BANs

Sample -BBF BANs:
  new_ban_name new_account_id        old_ban_sfid  \
0  A117300-BBF         117300  aA30B00000000o4SAA   
1  A117301-BBF         117301  aA30B00000000iESAQ   
2  A117302-BBF         117302  aA30B00000000fzSAA   
3  A117303-BBF         117303  aA30B00000000nXSAQ   
4  A117304-BBF         117304  aA30B00000000nJSAQ   
5  A117305-BBF         117305  aA30B00000000nOSAQ   
6  A117306-BBF         117306  aA30B00000178WDSAY   
7  A117307-BBF         117307  aA33g0000008OPbCAM   
8  A117308-BBF         117308  aA30B00000000loSAA   
9  A117309-BBF         117309  aA30B00000000gkSAA   

                             account_name__c  
0      AIS Realtime Locations Solutions Inc.  
1                                    Peckham  
2                              Ingham County  
3                         Buckeye Telesystem  
4                                    LakeNet  
5                                     Grid 4  
6  CCI Systems 

In [4]:
# === STEP 2: Get old BAN info for the mapping ===

old_ban_sfids = list(bbf_bans_df["old_ban_sfid"].dropna())
old_ban_sfids_str = "','".join(old_ban_sfids)

old_bans_sql = f"""
SELECT 
    sfid AS old_ban_sfid,
    name AS old_ban_name,
    account_number__c AS old_account_id
FROM sfprod.billing_invoice__c
WHERE sfid IN ('{old_ban_sfids_str}')
"""

print("Querying old BAN info...")
with conn.cursor() as cur:
    cur.execute(old_bans_sql)
    old_bans_rows = cur.fetchall()

old_bans_df = pd.DataFrame(old_bans_rows)
print(f"✅ Found {len(old_bans_df)} old BANs")

# Add old BAN info to mapping
old_ban_sfid_to_name = dict(
    zip(old_bans_df["old_ban_sfid"], old_bans_df["old_ban_name"])
)
old_ban_sfid_to_account_id = dict(
    zip(old_bans_df["old_ban_sfid"], old_bans_df["old_account_id"])
)

Querying old BAN info...
✅ Found 2505 old BANs


In [5]:
# === STEP 3: Get all orders that should move to -BBF BANs ===

status_str = "','".join(ACTIVE_STATUSES)

orders_to_move_sql = f"""
SELECT 
    o.sfid AS order_sfid,
    o.ordernumber,
    o.name AS order_name,
    o.service_id__c,
    o.status,
    o.billing_invoice__c AS current_ban_sfid,
    bi.name AS current_ban_name,
    bi.account_number__c AS current_account_id,
    o.oss_order__c,
    o.project_group__c,
    o.dimension_4_market__c AS market,
    o.accountid,
    a.name AS account_name,
    o.service_start_date__c,
    o.sof_mrc__c
FROM sfprod."order" o
JOIN sfprod.billing_invoice__c bi ON o.billing_invoice__c = bi.sfid
LEFT JOIN sfprod.account a ON o.accountid = a.sfid
WHERE o.status IN ('{status_str}')
  AND (o.project_group__c IS NULL OR o.project_group__c NOT LIKE '%{WD_PROJECT_GROUP}%')
  AND o.billing_invoice__c IS NOT NULL
ORDER BY bi.name, o.ordernumber;
"""

print("Querying orders to move (no PA MARKET DECOM)...")
with conn.cursor() as cur:
    cur.execute(orders_to_move_sql)
    all_bbf_orders_rows = cur.fetchall()

all_bbf_orders_df = pd.DataFrame(all_bbf_orders_rows)
print(f"✅ Found {len(all_bbf_orders_df):,} total BBF orders (no PA MARKET DECOM)")

# Filter to only orders on BANs that have a -BBF counterpart
orders_to_move_df = all_bbf_orders_df[
    all_bbf_orders_df["current_ban_sfid"].isin(old_to_new_ban_sfid.keys())
].copy()

print(
    f"✅ {len(orders_to_move_df):,} orders are on BANs with -BBF counterpart (will move)"
)

# Orders on BANs without -BBF counterpart
orders_no_bbf_ban = all_bbf_orders_df[
    ~all_bbf_orders_df["current_ban_sfid"].isin(old_to_new_ban_sfid.keys())
].copy()

print(f"⚠️ {len(orders_no_bbf_ban):,} orders are on BANs WITHOUT -BBF counterpart")

Querying orders to move (no PA MARKET DECOM)...


UndefinedColumn: column o.sof_mrc__c does not exist
LINE 17:     o.sof_mrc__c
             ^


In [ ]:
# === STEP 4: Add new BAN info to orders ===

orders_to_move_df["new_ban_sfid"] = orders_to_move_df["current_ban_sfid"].map(
    old_to_new_ban_sfid
)
orders_to_move_df["new_ban_name"] = orders_to_move_df["current_ban_sfid"].map(
    old_to_new_ban_name
)
orders_to_move_df["new_account_id"] = orders_to_move_df["current_ban_sfid"].map(
    old_to_new_account_id
)

print("Added new BAN mapping to orders")
print(f"\nSample orders to move:")
print(
    orders_to_move_df[
        [
            "ordernumber",
            "current_ban_name",
            "new_ban_name",
            "current_account_id",
            "new_account_id",
        ]
    ].head(10)
)

In [ ]:
# === STEP 5: Check OSS Order linkage ===

# Orders with OSS_Order__c
orders_with_oss = orders_to_move_df[orders_to_move_df["oss_order__c"].notna()].copy()
orders_without_oss = orders_to_move_df[orders_to_move_df["oss_order__c"].isna()].copy()

print(f"\n=== OSS Order Linkage ===")
print(f"✅ Orders WITH OSS_Order__c: {len(orders_with_oss):,}")
print(f"❌ Orders WITHOUT OSS_Order__c: {len(orders_without_oss):,}")

if len(orders_without_oss) > 0:
    print(
        f"\n⚠️ WARNING: {len(orders_without_oss):,} orders cannot be updated in OSS (no OSS_Order__c link)"
    )

In [ ]:
# === STEP 6: Verify OSS orders exist and get current account_id ===

oss_order_ids = orders_with_oss["oss_order__c"].dropna().astype(int).tolist()
missing_oss_ids = set()

if len(oss_order_ids) > 0:
    # Query in batches
    batch_size = 5000
    oss_orders_list = []

    print(f"\nVerifying {len(oss_order_ids):,} OSS orders exist...")

    for i in range(0, len(oss_order_ids), batch_size):
        batch = oss_order_ids[i : i + batch_size]
        batch_str = ",".join(map(str, batch))

        oss_orders_sql = f"""
        SELECT 
            order_id,
            account_id AS current_oss_account_id,
            order_status
        FROM om.orders
        WHERE order_id IN ({batch_str})
        """

        with oconn.cursor() as ocur:
            ocur.execute(oss_orders_sql)
            batch_rows = ocur.fetchall()

        batch_df = pd.DataFrame(batch_rows)
        oss_orders_list.append(batch_df)
        print(f"  Batch {i//batch_size + 1}: {len(batch_df)} found")

    oss_orders_df = pd.concat(oss_orders_list, ignore_index=True)
    print(f"\n✅ Found {len(oss_orders_df):,} OSS orders")

    # Check for missing OSS orders
    found_oss_ids = set(oss_orders_df["order_id"].tolist())
    missing_oss_ids = set(oss_order_ids) - found_oss_ids

    if missing_oss_ids:
        print(
            f"\n❌ WARNING: {len(missing_oss_ids)} OSS_Order__c values do not exist in OSS!"
        )
    else:
        print(f"\n✅ All OSS_Order__c values found in OSS")
else:
    print("No OSS orders to verify")
    oss_orders_df = pd.DataFrame()

In [ ]:
# === STEP 7: Build complete order move mapping ===

if len(oss_orders_df) > 0:
    orders_with_oss["oss_order__c_int"] = orders_with_oss["oss_order__c"].astype(int)
    orders_with_oss = orders_with_oss.merge(
        oss_orders_df[["order_id", "current_oss_account_id"]],
        left_on="oss_order__c_int",
        right_on="order_id",
        how="left",
    )
    orders_with_oss.drop(
        columns=["order_id", "oss_order__c_int"], inplace=True, errors="ignore"
    )

print("\n=== Complete Order Move Mapping ===")
print(f"Orders with full mapping (SF + OSS): {len(orders_with_oss):,}")

In [ ]:
# === STEP 8: Summary Statistics ===

print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)

print(f"\n{'Category':<50} {'Count':>10}")
print("-" * 65)
print(f"{'New -BBF BANs':<50} {len(bbf_bans_df):>10,}")
print(f"{'Total BBF orders (no PA MARKET DECOM)':<50} {len(all_bbf_orders_df):>10,}")
print(f"{'Orders on BANs WITH -BBF (will move)':<50} {len(orders_to_move_df):>10,}")
print(f"{'Orders on BANs WITHOUT -BBF':<50} {len(orders_no_bbf_ban):>10,}")
print()
print(f"{'Orders WITH OSS_Order__c':<50} {len(orders_with_oss):>10,}")
print(f"{'Orders WITHOUT OSS_Order__c':<50} {len(orders_without_oss):>10,}")

if len(oss_orders_df) > 0:
    print(f"{'OSS orders verified exist':<50} {len(oss_orders_df):>10,}")
    print(f"{'OSS orders NOT found':<50} {len(missing_oss_ids):>10,}")

print()
print(f"{'SF ORDERS TO UPDATE (Billing_Invoice__c)':<50} {len(orders_to_move_df):>10,}")
print(f"{'OSS ORDERS TO UPDATE (account_id)':<50} {len(orders_with_oss):>10,}")

In [ ]:
# === STEP 9: Execute Updates (or Dry Run) ===

# Track results
sf_results = []
oss_results = []

if DRY_RUN:
    print("\n" + "=" * 70)
    print("DRY RUN - No changes will be made")
    print("=" * 70)

    # Simulate SF updates
    for _, row in orders_to_move_df.iterrows():
        sf_results.append(
            {
                "order_sfid": row["order_sfid"],
                "ordernumber": row["ordernumber"],
                "service_id": row["service_id__c"],
                "account_name": row["account_name"],
                "current_ban_sfid": row["current_ban_sfid"],
                "current_ban_name": row["current_ban_name"],
                "new_ban_sfid": row["new_ban_sfid"],
                "new_ban_name": row["new_ban_name"],
                "status": "WOULD_UPDATE",
                "error": None,
            }
        )

    # Simulate OSS updates
    for _, row in orders_with_oss.iterrows():
        oss_results.append(
            {
                "oss_order_id": int(row["oss_order__c"]),
                "ordernumber": row["ordernumber"],
                "current_oss_account_id": row.get("current_oss_account_id", ""),
                "new_account_id": row["new_account_id"],
                "status": "WOULD_UPDATE",
                "error": None,
            }
        )

    print(f"\nWould update {len(sf_results):,} SF orders")
    print(f"Would update {len(oss_results):,} OSS orders")

else:
    print("\n" + "=" * 70)
    print("LIVE RUN - Updating orders")
    print("=" * 70)

    # === SF Updates ===
    print(f"\n--- Updating {len(orders_to_move_df):,} SF Orders ---")

    sf_update_count = 0
    sf_error_count = 0

    for idx, row in orders_to_move_df.iterrows():
        result = {
            "order_sfid": row["order_sfid"],
            "ordernumber": row["ordernumber"],
            "service_id": row["service_id__c"],
            "account_name": row["account_name"],
            "current_ban_sfid": row["current_ban_sfid"],
            "current_ban_name": row["current_ban_name"],
            "new_ban_sfid": row["new_ban_sfid"],
            "new_ban_name": row["new_ban_name"],
            "status": None,
            "error": None,
        }

        try:
            es_sf.Order.update(
                row["order_sfid"], {"Billing_Invoice__c": row["new_ban_sfid"]}
            )
            result["status"] = "UPDATED"
            sf_update_count += 1
        except Exception as e:
            result["status"] = "ERROR"
            result["error"] = str(e)
            sf_error_count += 1

        sf_results.append(result)

        if (sf_update_count + sf_error_count) % 500 == 0:
            print(f"  Processed {sf_update_count + sf_error_count:,} SF orders...")

    print(
        f"\n✅ SF Updates complete: {sf_update_count:,} updated, {sf_error_count:,} errors"
    )

    # === OSS Updates ===
    print(f"\n--- Updating {len(orders_with_oss):,} OSS Orders ---")

    oss_update_count = 0
    oss_error_count = 0

    for idx, row in orders_with_oss.iterrows():
        result = {
            "oss_order_id": int(row["oss_order__c"]),
            "ordernumber": row["ordernumber"],
            "current_oss_account_id": row.get("current_oss_account_id", ""),
            "new_account_id": row["new_account_id"],
            "status": None,
            "error": None,
        }

        try:
            with oconn.cursor() as ocur:
                ocur.execute(
                    """
                    UPDATE om.orders
                    SET account_id = %s
                    WHERE order_id = %s
                """,
                    (row["new_account_id"], int(row["oss_order__c"])),
                )
            oconn.commit()
            result["status"] = "UPDATED"
            oss_update_count += 1
        except Exception as e:
            result["status"] = "ERROR"
            result["error"] = str(e)
            oconn.rollback()
            oss_error_count += 1

        oss_results.append(result)

        if (oss_update_count + oss_error_count) % 500 == 0:
            print(f"  Processed {oss_update_count + oss_error_count:,} OSS orders...")

    print(
        f"\n✅ OSS Updates complete: {oss_update_count:,} updated, {oss_error_count:,} errors"
    )

In [ ]:
# === STEP 10: Create Output Excel ===

wb = Workbook()

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill_blue = PatternFill("solid", fgColor="4472C4")
header_fill_green = PatternFill("solid", fgColor="70AD47")
header_fill_orange = PatternFill("solid", fgColor="ED7D31")
header_fill_red = PatternFill("solid", fgColor="C00000")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def style_header(ws, headers, fill_color):
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = fill_color
        cell.alignment = header_alignment
        cell.border = thin_border


def auto_width(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column].width = min(max_length + 2, 50)


# Status colors
status_colors = {
    "WOULD_UPDATE": "C6EFCE",  # Light green
    "UPDATED": "C6EFCE",  # Light green
    "ERROR": "FF6666",  # Red
}

# === Sheet 1: Summary ===
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["ES Order Move to BBF BANs (v2)"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws1.append(["Generated:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append([])
ws1.append(["BAN Mapping:"])
ws1["A6"].font = Font(bold=True)
ws1.append(["  New -BBF BANs:", len(bbf_bans_df)])
ws1.append([])
ws1.append(["Order Counts:"])
ws1["A9"].font = Font(bold=True)
ws1.append(["  Total BBF orders (no PA MARKET DECOM):", len(all_bbf_orders_df)])
ws1.append(["  Orders on BANs WITH -BBF:", len(orders_to_move_df)])
ws1.append(["  Orders on BANs WITHOUT -BBF:", len(orders_no_bbf_ban)])
ws1.append([])
ws1.append(["OSS Linkage:"])
ws1["A14"].font = Font(bold=True)
ws1.append(["  Orders WITH OSS_Order__c:", len(orders_with_oss)])
ws1.append(["  Orders WITHOUT OSS_Order__c:", len(orders_without_oss)])
ws1.append([])
ws1.append(["Results:"])
ws1["A18"].font = Font(bold=True)

# Count statuses
sf_status_counts = {}
for r in sf_results:
    sf_status_counts[r["status"]] = sf_status_counts.get(r["status"], 0) + 1
oss_status_counts = {}
for r in oss_results:
    oss_status_counts[r["status"]] = oss_status_counts.get(r["status"], 0) + 1

ws1.append(["  SF Orders:"])
for status, count in sf_status_counts.items():
    ws1.append([f"    {status}:", count])
ws1.append(["  OSS Orders:"])
for status, count in oss_status_counts.items():
    ws1.append([f"    {status}:", count])

# === Sheet 2: SF Update Results ===
ws2 = wb.create_sheet("SF Update Results")
headers2 = [
    "Order SFID",
    "Order Number",
    "Service ID",
    "Account Name",
    "Current BAN SFID",
    "Current BAN Name",
    "New BAN SFID",
    "New BAN Name",
    "Status",
    "Error",
]
ws2.append(headers2)
style_header(ws2, headers2, header_fill_green)

for row_idx, r in enumerate(sf_results, 2):
    ws2.append(
        [
            r["order_sfid"],
            r["ordernumber"],
            r["service_id"],
            r["account_name"],
            r["current_ban_sfid"],
            r["current_ban_name"],
            r["new_ban_sfid"],
            r["new_ban_name"],
            r["status"],
            r["error"],
        ]
    )
    # Color by status
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers2) + 1):
        ws2.cell(row=row_idx, column=col).fill = PatternFill(
            "solid", fgColor=fill_color
        )

ws2.freeze_panes = "A2"
auto_width(ws2)

# === Sheet 3: OSS Update Results ===
ws3 = wb.create_sheet("OSS Update Results")
headers3 = [
    "OSS Order ID",
    "Order Number",
    "Current OSS Account ID",
    "New Account ID",
    "Status",
    "Error",
]
ws3.append(headers3)
style_header(ws3, headers3, header_fill_blue)

for row_idx, r in enumerate(oss_results, 2):
    ws3.append(
        [
            r["oss_order_id"],
            r["ordernumber"],
            r["current_oss_account_id"],
            r["new_account_id"],
            r["status"],
            r["error"],
        ]
    )
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers3) + 1):
        ws3.cell(row=row_idx, column=col).fill = PatternFill(
            "solid", fgColor=fill_color
        )

ws3.freeze_panes = "A2"
auto_width(ws3)

# === Sheet 4: BAN Mapping ===
ws4 = wb.create_sheet("BAN Mapping")
headers4 = [
    "Old BAN SFID",
    "Old BAN Name",
    "Old Account ID",
    "New BAN SFID",
    "New BAN Name",
    "New Account ID",
    "Account Name",
]
ws4.append(headers4)
style_header(ws4, headers4, header_fill_blue)

for _, row in bbf_bans_df.iterrows():
    ws4.append(
        [
            row["old_ban_sfid"],
            old_ban_sfid_to_name.get(row["old_ban_sfid"], ""),
            old_ban_sfid_to_account_id.get(row["old_ban_sfid"], ""),
            row["new_ban_sfid"],
            row["new_ban_name"],
            row["new_account_id"],
            row["account_name__c"],
        ]
    )

ws4.freeze_panes = "A2"
auto_width(ws4)

# === Sheet 5: Missing OSS Link ===
ws5 = wb.create_sheet("Missing OSS Link")
headers5 = [
    "Order Number",
    "Service ID",
    "Status",
    "Account Name",
    "Current BAN Name",
    "New BAN Name",
    "Market",
]
ws5.append(headers5)
style_header(ws5, headers5, header_fill_orange)

for _, row in orders_without_oss.iterrows():
    ws5.append(
        [
            row["ordernumber"],
            row["service_id__c"],
            row["status"],
            row["account_name"],
            row["current_ban_name"],
            row["new_ban_name"],
            row["market"],
        ]
    )

ws5.freeze_panes = "A2"
auto_width(ws5)

# === Sheet 6: No BBF BAN Available ===
ws6 = wb.create_sheet("No BBF BAN Available")
headers6 = [
    "Order Number",
    "Service ID",
    "Status",
    "Account Name",
    "Current BAN Name",
    "Current BAN SFID",
    "Market",
]
ws6.append(headers6)
style_header(ws6, headers6, header_fill_red)

for _, row in orders_no_bbf_ban.iterrows():
    ws6.append(
        [
            row["ordernumber"],
            row["service_id__c"],
            row["status"],
            row["account_name"],
            row["current_ban_name"],
            row["current_ban_sfid"],
            row["market"],
        ]
    )

ws6.freeze_panes = "A2"
auto_width(ws6)

# === Sheet 7: Errors Only ===
ws7 = wb.create_sheet("Errors")
headers7 = ["Type", "Order ID", "Order Number", "Error"]
ws7.append(headers7)
style_header(ws7, headers7, header_fill_red)

for r in sf_results:
    if r["status"] == "ERROR":
        ws7.append(["SF", r["order_sfid"], r["ordernumber"], r["error"]])
for r in oss_results:
    if r["status"] == "ERROR":
        ws7.append(["OSS", r["oss_order_id"], r["ordernumber"], r["error"]])

ws7.freeze_panes = "A2"
auto_width(ws7)

# Save
wb.save(output_file)
print(f"\n✅ Output saved to: {output_file}")

In [ ]:
# === Cleanup ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")

if DRY_RUN:
    print("\n🔄 DRY RUN complete. No orders were updated.")
    print(f"Review the output file: {output_file}")
    print("\nTo run for real, set DRY_RUN = False and re-run the notebook.")
else:
    print(f"\n✅ LIVE RUN complete.")
    print(f"Output saved to: {output_file}")